# 02 — Quantum Circuit Tutorial

Interactive exploration of the quantum building blocks used in our hybrid models:

1. **Data encodings** — Angle, Amplitude, IQP
2. **Ansatz circuits** — StronglyEntangling, HardwareEfficient, BasicEntangler
3. **Entanglement patterns** — Full, Linear, Circular
4. **Circuit diagrams** and parameter analysis

In [ ]:
import sys
from pathlib import Path

PROJECT_ROOT = Path.cwd().parent if Path.cwd().name == 'notebooks' else Path.cwd()
sys.path.insert(0, str(PROJECT_ROOT))

import numpy as np
import pennylane as qml
import matplotlib.pyplot as plt

from src.quantum.circuits import build_ansatz, get_weight_shape
from src.quantum.encodings import angle_encoding, amplitude_encoding, iqp_encoding
from src.quantum.entanglement import get_entanglement_pairs, count_cnot_gates
from src.quantum.measurements import measure_expectations

print(f'PennyLane {qml.__version__}')

## 1. Data Encoding Strategies

Encoding classical data $\mathbf{x} \in \mathbb{R}^n$ into an $n$-qubit quantum state $|\psi(\mathbf{x})\rangle$.

In [ ]:
n_qubits = 4
dev = qml.device('default.qubit', wires=n_qubits)
sample_data = np.array([0.5, -0.3, 0.8, -0.1])

# Angle encoding
@qml.qnode(dev)
def angle_circuit(x):
    angle_encoding(x, n_qubits, rotation_axes=['Y'])
    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]

print('Angle encoding circuit:')
print(qml.draw(angle_circuit)(sample_data))
print(f'\nOutputs: {angle_circuit(sample_data)}')

## 2. Variational Ansatz Circuits

The trainable part of the VQC, parameterised by weights $\boldsymbol{\theta}$.

In [ ]:
ansatzes = ['strongly_entangling', 'hardware_efficient', 'basic_entangler']

for ansatz_type in ansatzes:
    n_layers = 2
    w_shape = get_weight_shape(ansatz_type, n_qubits, n_layers)
    weights = np.random.randn(*w_shape) * 0.5
    
    @qml.qnode(dev)
    def circuit(w, at=ansatz_type):
        build_ansatz(w, n_qubits, n_layers, at, 'full')
        return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]
    
    print(f'\n=== {ansatz_type} (shape={w_shape}) ===')
    print(qml.draw(circuit)(weights))
    print(f'Parameters: {np.prod(w_shape)}')

## 3. Entanglement Pattern Comparison

In [ ]:
patterns = ['full', 'linear', 'circular']

for pattern in patterns:
    pairs = get_entanglement_pairs(n_qubits, pattern)
    n_cnots = count_cnot_gates(n_qubits, pattern)
    print(f'{pattern:10s}: {n_cnots} CNOTs — pairs: {pairs}')

## 4. Full VQC: Encoding + Ansatz + Measurement

In [ ]:
from src.models.quantum_layers import PennyLaneQuantumLayer
import torch

qlayer = PennyLaneQuantumLayer(
    n_qubits=4, n_layers=2,
    encoding_type='angle',
    ansatz_type='strongly_entangling',
    entanglement='full',
    n_outputs=2,
    diff_method='backprop',
)

# Draw circuit
print('Full VQC circuit diagram:')
print(qlayer.draw())
print(f'\nTrainable quantum parameters: {qlayer.count_parameters()}')

# Forward pass
x = torch.randn(1, 4)
out = qlayer(x)
print(f'\nInput shape:  {x.shape}')
print(f'Output shape: {out.shape}')
print(f'Output values: {out.detach().numpy()}')

## 5. Parameter Count Summary

In [ ]:
# Compare parameter counts across ansatzes
results = []
for ansatz in ansatzes:
    for depth in [1, 2, 3, 4, 5]:
        shape = get_weight_shape(ansatz, n_qubits, depth)
        results.append({'ansatz': ansatz, 'depth': depth, 'params': int(np.prod(shape))})

fig, ax = plt.subplots(figsize=(8, 5))
for ansatz in ansatzes:
    data = [r for r in results if r['ansatz'] == ansatz]
    ax.plot([r['depth'] for r in data], [r['params'] for r in data],
            marker='o', label=ansatz, linewidth=2)
ax.set_xlabel('Circuit Depth')
ax.set_ylabel('Trainable Parameters')
ax.set_title('Parameter Count vs Circuit Depth')
ax.legend()
plt.tight_layout()
plt.show()